In [0]:
%run "../commons/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_mount_path}/race_results")

In [0]:
from pyspark.sql.functions import sum, count, col, when

driver_standings_df = race_results_df.groupBy("race_year","driver_name","driver_nationality","team")\
    .agg(sum("points").alias("total_points"),
         count(when(col("position") == 1, True)).alias("wins"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc, asc

driver_rank = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))

driver_rank_df = driver_standings_df.withColumn("rank", rank().over(driver_rank))

In [0]:
#driver_rank_df.write.parquet(f"{presentation_mount_path}/driver_standings", mode="overwrite")
driver_rank_df.write.mode("overwrite").format("parquet").saveAsTable("presentation.driver_standings")